# all()方法

```python
# all()方法，当a，b，c全为真，all()返回真
all([a,b,c])
```

- 接收数据
- 校验数据完整性
- 校验数据格式
- 与数据库比对
- 发送邮件
- 返回数据

# Django认证系统
- 添加用户：

```
user = User.object.create_user(username, email, password)
user.is_active = 0
user.save()
```

# 类视图
- 对请求方式分类处理，使用类视图
- 变量:类名，指定类型
```python
# 导入django.view.generic.View
class RegisterView(View):
    def get(self, request:HttpRequest):
        pass
    def post(self, request):
        pass
# url指向RegisterView.as_View()会对请求方式进行分发，未定义的请求方式返回方法未找到
```

# itsdangerous模块加密
```python
from itsdangerous import TimedJSONWebSignatureSerializer as Se

se = Se('key',3600)  # 设置key和过期时间（秒）
info = {'liduo':1}
res = se.dumps(info)  # 加密
ans = res.load(res)  # 解密
```

# 发送邮件
- 获取SMTP服务器

```python
# settings.py文件

# 发送邮件配置
EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'
# smpt服务地址
EMAIL_HOST = 'smtp.163.com'
EMAIL_PORT = 25
# 发送邮件的邮箱
EMAIL_HOST_USER = 'smartli_it@163.com'
# 在邮箱中设置的客户端授权密码
EMAIL_HOST_PASSWORD = 'smartli123'
# 收件人看到的发件人
EMAIL_FROM = 'Python<smartli_it@163.com>'
```

- from django.core.mail import send_mail
- send_mail(subject, message, from_email, recive_list,html_message=...)：标题，正文，发件人，收件人列表
- message：字符串
- html_message：发送HTML标签

# celery异步发送邮件
- 任务发出者-->任务队列（中间人broker）-->任务处理者（worker）
- 中间人：RabbitMQ或Redis
- 发出者：项目代码，加入队列
- 处理者：监听队列，执行代码
- 1.安装包：celery
- 2.新建文件celery_tasks/tasks.py保存任务
- 3.tasks.py写代码

```python
# 使用celery
from django.core.mail import send_mail
from django.conf import settings
from celery import Celery
import time

# 在任务处理者一端加这几句
# import os
# import django
# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dailyfresh.settings")
# django.setup()

# 创建一个Celery类的实例对象
app = Celery('celery_task.task', broker='redis://ip:port/数据库')

# app = Celery('celery_tasks.tasks', broker='redis://172.16.179.130:6379/8')


# 定义任务函数
@app.task
def send_register_active_email(to_email, username, token):
    '''发送激活邮件'''
    # 组织邮件信息
    subject = '欢迎信息'
    message = ''
    sender = settings.EMAIL_FROM
    receiver = [to_email]
    html_message = '<h1>%s, 欢迎您成为注册会员</h1>请点击下面链接激活您的账户<br/><a href="http://127.0.0.1:8000/user/active/%s">http://127.0.0.1:8000/user/active/%s</a>' % (username, token, token)

    send_mail(subject, message, sender, receiver, html_message=html_message)
    time.sleep(5)
```

- 4.发布任务

```python
send_register_active_email.delay(email, username, token)
```

- 5.启动worker

```shell
cd /.../django项目
celery -A celery_tasks.tasks worker -l info
```